In [1]:
import sqlite3
import pandas as pd
from datetime import datetime

In [3]:
# Connect to SQLite DB
conn = sqlite3.connect("../weather.db")
cursor = conn.cursor()

In [4]:
# Create tables with foreign keys and dimensions
cursor.executescript("""
-- Dimension: Time
CREATE TABLE IF NOT EXISTS dim_tiempo (
    fecha_id TEXT PRIMARY KEY,
    fecha TEXT,
    año INTEGER,
    mes INTEGER,
    día INTEGER,
    nombre_mes TEXT,
    trimestre INTEGER
);

-- Dimension: Station
CREATE TABLE IF NOT EXISTS dim_estacion (
    estacion_id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre_estacion TEXT,
    provincia TEXT,
    latitud REAL,
    longitud REAL,
    UNIQUE(nombre_estacion, provincia)
);

-- Dimension: Wind Direction
CREATE TABLE IF NOT EXISTS dim_direccion_viento (
    wdir_id INTEGER PRIMARY KEY AUTOINCREMENT,
    direccion_cardinal TEXT UNIQUE
);

-- Fact Table
CREATE TABLE IF NOT EXISTS hechos_clima (
    id_hecho INTEGER PRIMARY KEY AUTOINCREMENT,
    fecha_id TEXT,
    estacion_id INTEGER,
    wdir_id INTEGER,
    tmax REAL,
    tmin REAL,
    tavg REAL,
    prcp REAL,
    wspd REAL,
    FOREIGN KEY (fecha_id) REFERENCES dim_tiempo(fecha_id),
    FOREIGN KEY (estacion_id) REFERENCES dim_estacion(estacion_id),
    FOREIGN KEY (wdir_id) REFERENCES dim_direccion_viento(wdir_id)
);

-- Indexes for optimization
CREATE INDEX IF NOT EXISTS idx_hechos_fecha_id ON hechos_clima(fecha_id);
CREATE INDEX IF NOT EXISTS idx_hechos_estacion_id ON hechos_clima(estacion_id);
CREATE INDEX IF NOT EXISTS idx_hechos_wdir_id ON hechos_clima(wdir_id);
""")

In [5]:
def insert_dim_tiempo(row):
    fecha = datetime.strptime(row["Date"], "%Y-%m-%d")
    fecha_id = row["Date"]
    año = fecha.year
    mes = fecha.month
    día = fecha.day
    nombre_mes = fecha.strftime("%B")
    trimestre = (mes - 1) // 3 + 1

    cursor.execute("""
    INSERT OR IGNORE INTO dim_tiempo (fecha_id, fecha, año, mes, día, nombre_mes, trimestre)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (fecha_id, row["Date"], año, mes, día, nombre_mes, trimestre))

    return fecha_id

In [11]:
def insert_dim_estacion(nombre, provincia):
    cursor.execute("""
    INSERT OR IGNORE INTO dim_estacion (nombre_estacion, provincia, latitud, longitud)
    VALUES (?, ?, NULL, NULL)
    """, (nombre, provincia))
    cursor.execute("""
    SELECT estacion_id FROM dim_estacion WHERE nombre_estacion = ? AND provincia = ?
    """, (nombre, provincia))
    return cursor.fetchone()[0]

In [12]:
def insert_dim_direccion_viento(direccion):
    cursor.execute("""
    INSERT OR IGNORE INTO dim_direccion_viento (direccion_cardinal)
    VALUES (?)
    """, (direccion,))
    cursor.execute("""
    SELECT wdir_id FROM dim_direccion_viento WHERE direccion_cardinal = ?
    """, (direccion,))
    return cursor.fetchone()[0]

In [10]:
# Load CSV
csv_path = '../datos_limpios/datos_climaticos_chiriqui_2024.csv'  # Replace with the path to your CSV
df = pd.read_csv(csv_path)
print(df)

           Date  Tmax  Tmin  Tavg  Prcp  Wspd         Wdir Estacion Provincia
0    2024-01-01  34.4  23.0  28.7   0.0  43.2        Norte    David   Chirqui
1    2024-01-02  34.4  23.0  28.7   0.0  43.2        Norte    David   Chirqui
2    2024-01-03  33.2  22.0  27.6   0.0  28.8          Sur    David   Chirqui
3    2024-01-04  34.0  22.6  28.3   0.0  30.2  Nor-Noreste    David   Chirqui
4    2024-01-05  34.2  22.6  28.4   0.0  34.9          Sur    David   Chirqui
..          ...   ...   ...   ...   ...   ...          ...      ...       ...
361  2024-12-27  28.0  24.0  26.0   0.7  13.4     Suroeste    David   Chirqui
362  2024-12-28  25.2  24.0  24.6  10.4   7.0          Sur    David   Chirqui
363  2024-12-29  27.0  23.8  25.4  10.3  12.9        Norte    David   Chirqui
364  2024-12-30  29.2  21.6  25.4   0.0  22.1     Suroeste    David   Chirqui
365  2024-12-31  30.8  22.4  26.6   0.0  25.6          Sur    David   Chirqui

[366 rows x 9 columns]


In [ ]:
# Insert data
for _, row in df.iterrows():
    fecha_id = insert_dim_tiempo(row)
    estacion_id = insert_dim_estacion(row["Estacion"], row["Provincia"])
    wdir_id = insert_dim_direccion_viento(row["Wdir"])

    cursor.execute("""
    INSERT INTO hechos_clima (
        fecha_id, estacion_id, wdir_id, tmax, tmin, tavg, prcp, wspd
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        fecha_id,
        estacion_id,
        wdir_id,
        row["Tmax"],
        row["Tmin"],
        row["Tavg"],
        row["Prcp"],
        row["Wspd"]
    ))

# Commit changes and close connection
conn.commit()
conn.close()